In [ ]:
!pip install pennylane
!pip install qiskit
!pip install qiskit_aer
!pip install gurobipy
!pip install tensorcircuit
!pip install quimb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/TRVQA

# List files to make sure we're in the expected directory.
# Your output will look different, showing your own Drive files here.
!ls
import sys
sys.path.append('/content/drive/My\ Drive/TRVQA')


<>:9: SyntaxWarning: invalid escape sequence '\ '
<>:9: SyntaxWarning: invalid escape sequence '\ '
/tmp/ipython-input-2251797223.py:9: SyntaxWarning: invalid escape sequence '\ '
  sys.path.append('/content/drive/My\ Drive/TRVQA')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/TRVQA
bench_10S_10I		       bench_out_comparison_gpu
bench_10S_10I_start	       bench_out_pl
benchmark		       bench_out_shot_diff
bench_out		       bench_out_shot_diff_2436
bench_out_100		       bench_out_shot_diff_24_real
bench_out_1000		       bench_out_shot_diff_36_real
bench_out_100_old	       bench_out_shot_diff_4816_real
bench_out_150		       bench_out_shot_diff_52
bench_out_2436		       bench_out_shot_diff_52_real
bench_out_4816		       bench_tsp_4816
bench_out_4816_lr_3	       bench_tsp_49
bench_out_4816_old	       circuit.py
bench_out_4816_sv_compare      ex_value_pots
bench_out_52		       gates
bench_out_52_lr		       hamiltonian
bench_out_52_memory	       __init__.py
bench_out_52_old	       measure
bench_out_52_opt	       optimization
bench_out_comparison	       __pycache__
bench_out_comparison_cpu_real  utils


In [ ]:
import sys
import os
import sys
import os
# Add the parent directory of TRVQA to sys.path
from measure.enums import MeasureMethod
from optimization.gradients.vanilla_parameter_shift import vanilla_parameter_shift
from optimization.optimization import minimize, minimize_custom
#from utils.maxcut import gengraph, create_hamiltonian, make_hamiltonian
import torch
from torch.autograd import Function
from qiskit.circuit.library import QAOAAnsatz
from qiskit.quantum_info import SparsePauliOp, Statevector
from qiskit import QuantumCircuit

import numpy as np
import torch
from circuit import Circuit

def compute_maxcut_value(bitstring, graph):
    """Computes the MaxCut value for a given bitstring solution."""
    cut_value = sum(w for (u,v), w in graph if bitstring[u] != bitstring[v])
    return cut_value

# for s in best_value:
#     print(compute_maxcut_value(s, g))

from pyparsing import Optional
import torch
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, Operator, SparsePauliOp
from optimization.optimization import minimize, minimize_custom
# TRVQA imports

from circuit import Circuit
from hamiltonian.hamiltonian import Hamiltonian
from measure.enums import MeasureMethod
torch.cuda.empty_cache()
import numpy as np
import torch
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, Operator, SparsePauliOp
import torch

from measure.enums import MeasureMethod
from optimization.optimization import minimize
from utils.maxcut import gengraph, create_hamiltonian,make_hamiltonian
from hamiltonian.hamiltonian import Hamiltonian
import numpy as np
import torch
from circuit import Circuit
import time


In [ ]:
# Load initial parameters
theta_init_by_seed = []
for i in range(10):
  a = torch.load(f"/content/drive/MyDrive/TRVQA/benchmark/theta_inits/theta_init_s{i}.pt")
  b = torch.load(f"/content/drive/MyDrive/TRVQA/benchmark/theta_inits/theta_init_s{i}_extra.pt")
  theta_init_by_seed.append(torch.cat([a,b]))

In [ ]:
import json
with open("benchmark/experiment_grid.json", "r") as f:
    loaded = json.load(f)

# Convert lists back to tuples if you want
def convert_lists_to_tuples(obj):
    if isinstance(obj, list):
        # Check if list looks like an edge [(0,1), w]
        if len(obj) == 2 and isinstance(obj[0], list) and isinstance(obj[1], (int, float)):
            return (tuple(obj[0]), obj[1])
        else:
            return [convert_lists_to_tuples(x) for x in obj]
    if isinstance(obj, dict):
        return {k: convert_lists_to_tuples(v) for k, v in obj.items()}
    return obj

graph_by_seed = [convert_lists_to_tuples(entry) for entry in loaded]

In [ ]:
import os, csv, json, time, uuid
from dataclasses import dataclass
from typing import Any, List, Optional

# 1) RunLog (same shape you specified)
# ────────────────────────────────────────────────────────────────────────────────
@dataclass
class RunLog:
    exp_values: List[float]        # e.g., energies (float-able)
    best_results: List[Any]        # optional bitstrings/ints/etc.
    iter_times: List[float]        # per-iteration wall times (s)
    final_theta: torch.Tensor      # final parameter vector (Tensor)

# ────────────────────────────────────────────────────────────────────────────────
# 2) Minimal CSV summary + iter I/O (only library, N, D)
# ────────────────────────────────────────────────────────────────────────────────
def _read_summary_slim(summary_csv: str) -> List[dict]:
    if not os.path.exists(summary_csv):
        return []
    with open(summary_csv, "r", newline="") as f:
        return list(csv.DictReader(f))

def _write_summary_header_if_missing_slim(summary_csv: str) -> None:
    if not os.path.exists(summary_csv):
        os.makedirs(os.path.dirname(summary_csv), exist_ok=True)
        with open(summary_csv, "w", newline="") as f:
            csv.writer(f).writerow([
                "run_id", "library", "N", "depth", "seed",
                "theta_path", "iter_csv"
            ])

def _find_run_row_slim(
    library: str, N: int, D: int,seed: int, summary_csv: str
) -> Optional[dict]:
    rows = _read_summary_slim(summary_csv)
    cand = [r for r in rows
            if r["library"] == library
            and int(r["N"]) == N
            and int(r["depth"]) == D
            and int(r['seed']) == seed]
    return cand[-1] if cand else None

def _read_last_iter(iter_csv_path: str) -> int:
    if not os.path.exists(iter_csv_path):
        return -1
    last = -1
    with open(iter_csv_path, "r", newline="") as f:
        r = csv.DictReader(f)
        for row in r:
            last = int(row["iter"])
    return last

def _overwrite_row_slim(updated_row: dict, summary_csv: str) -> None:
    rows = _read_summary_slim(summary_csv)
    for i, r in enumerate(rows):
        if r["run_id"] == updated_row["run_id"]:
            rows[i] = updated_row
            break
    with open(summary_csv, "w", newline="") as f:
        w = csv.DictWriter(f, fieldnames=["run_id","library","N","depth","seed", "theta_path","iter_csv"])
        w.writeheader()
        w.writerows(rows)

# ────────────────────────────────────────────────────────────────────────────────
# 3) Append-in-place (SLIM)
# ────────────────────────────────────────────────────────────────────────────────
def append_run_inplace_slim(
    runlog: RunLog,
    *,
    library: str,            # e.g., "tensorcircuit"
    N: int,
    D: int,
    seed: int,
    outdir: str = "bench_out_comparison",
    summary_csv: str = "bench_out_comparison/summary.csv",
):
    """
    Append iterations to the (library, N, D) run; create it if missing.
    Only keeps minimal fields: library, N, depth, theta_path, iter_csv.
    """
    os.makedirs(outdir, exist_ok=True)
    _write_summary_header_if_missing_slim(summary_csv)

    row = _find_run_row_slim(library, N, D, seed, summary_csv)
    if row is None:
        run_id = str(uuid.uuid4())[:8]
        theta_path = os.path.join(outdir, f"theta_{run_id}.pt")
        iter_csv_path = os.path.join(outdir, f"iter_{run_id}.csv")
        # create iter CSV with header
        with open(iter_csv_path, "w", newline="") as f:
            w = csv.writer(f); w.writerow(["iter","exp_value","best_result","iter_time"])
        # add to summary
        with open(summary_csv, "a", newline="") as f:
            w = csv.writer(f)
            w.writerow([run_id, library, N, D, seed, theta_path, iter_csv_path])
        start_iter = 0
    else:
        run_id      = row["run_id"]
        theta_path  = row["theta_path"]
        iter_csv_path = row["iter_csv"]
        start_iter = _read_last_iter(iter_csv_path) + 1

    # Append iterations
    with open(iter_csv_path, "a", newline="") as f:
        w = csv.writer(f)
        for i, (e, b, t) in enumerate(zip(runlog.exp_values, runlog.best_results, runlog.iter_times)):
            w.writerow([start_iter + i, e, json.dumps(b), t])

    # Save final theta
    torch.save(runlog.final_theta.detach().cpu(), theta_path)

    return {
        "run_id": run_id,
        "theta_path": theta_path,
        "iter_csv": iter_csv_path,
        "start_iter": start_iter,
    }


In [ ]:
!pip install Ipython --upgrade
%load_ext autoreload
%autoreload 2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.2/616.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: traitlets
    Found existing installation: traitlets 5.7.1
    Uninstalling traitlets-5.7.1:
      Successfully uninstalled traitlets-5.7.1
  Attempting uninstall: Ipython
    Found existing installation: ipython 7.34.0
    Uninstalling ipython-7.34.0:
      Successfully uninstalled ipython-7.34.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipython==7.34.0, but you have ipython 9.6.0 which is incompatible.


In [ ]:
from tensorcircuit import backends

In [ ]:
import numpy as np
import warnings
import tensorcircuit as tc
import torch

import tensorcircuit as tc
# make sure you're on torch backend *before* building circuits
tc.set_backend("pytorch")

def build_ansatz_tc_pytorch(theta, N, D, use_mps):
    # theta: torch.Tensor with requires_grad=True
    th = theta.reshape(-1)  # flat view (no copy)

    c = tc.MPSCircuit(N) if use_mps else tc.Circuit(N)
    if use_mps:
        c.set_split_rules({"max_singular_values": 10})  # optional; beware aggressive truncation

    k = 0
    for _ in range(D):
        for i in range(N):
            # ❌ was: c.rx(i, theta=theta[k].item())  -> breaks graph
            c.rx(i, theta=th[k])                      # ✅ keep tensor
            k += 1
        for i in range(N):
            c.rz(i, theta=th[k])                      # ✅ keep tensor
            k += 1
        for i in range(N - 1):
            c.cx(i, i + 1)
        c.cx(N - 1, 0)
    return c



In [ ]:
import numpy as np
import torch
import math
import tensorcircuit as tc
K = tc.set_backend("pytorch")  # 1) ensure torch backend
# If you're using TensorCircuit:

# ────────────────────────────────────────────────────────────────────────────────

# ────────────────────────────────────────────────────────────────────────────────
# 4) Your benchmark: now returns RunLog and (optionally) saves via slim appender
# ────────────────────────────────────────────────────────────────────────────────
def benchmark_tensorcircuit_tc_pytorch(
    graph, D,seed, steps, use_mps, device, theta, lr=0.1,
    *,
    library_name: str = "tensorcircuit",
    save: bool = True,
    outdir: str = "bench_out_comparison"
):
    """
    Runs a simple VQE loop in TensorCircuit and collects a RunLog.
    If save=True, appends to (library_name, N, D) run using the slim CSV logger.
    """
    # infer N from edges
    N = max(max(u, v) for (u, v), _ in graph) + 1
    print(f"\n🚀 TensorCircuit VQE | N={N} | {'matrix_product_state' if use_mps else 'full'} Circuit,  {'CPU' if device == 'cpu' else 'GPU'} backend")

    # build Ising-like H from edges list -> list[(weight, mask)]
    hm_terms = create_hamiltonian(N, graph)  # expects same shape as your original helper
    theta = theta.detach().clone().requires_grad_(True)  #
    # params
    num_params = D * 2 * N
    # theta_np = np.random.randn(num_params)
    # theta = tc.backend.convert_to_tensor(theta_np)
    def energy_from_theta(th):
      # returns a real torch scalar
      energy = torch.zeros((), dtype=th.real.dtype, device=th.device)
      c = build_ansatz_tc_pytorch(th, N, D, use_mps=True)  # stays MPS
      for weight, mask in hm_terms:
          z_idx = [i for i, v in enumerate(mask) if v]
          if z_idx:
              e = c.expectation_ps(z=z_idx)
              energy = energy + torch.as_tensor(weight, dtype=energy.dtype, device=th.device) * torch.real(e)
          else:
              energy = energy + torch.as_tensor(weight, dtype=energy.dtype, device=th.device)
      return energy

    """
    RuntimeError: svd_backward: The singular vectors in the
     complex case are specified up to multiplication by e^{i phi}.
     The specified loss function depends on this phase term, making it ill-defined.
    """
    # ── Ansatz + loss
    @torch.no_grad()
    def parameter_shift_grad(th, shift=math.pi/2):
        """
        Standard parameter-shift for gates with generator eigenvalues ±1/2 (RX/RZ, etc.).
        Returns grad with same shape as th. Uses 2 evals per parameter.
        """
        th = th.detach().clone().requires_grad_(False)
        grad = torch.zeros_like(th, dtype=th.dtype)
        # vectorized over parameters via in-place add/sub per index
        for i in range(th.numel()):
            th[i] += shift
            Ep = energy_from_theta(th)
            th[i] -= 2*shift
            Em = energy_from_theta(th)
            th[i] += shift  # restore
            # (Ep - Em) / 2  for generators with ±1/2 spectrum (RX/RZ)
            grad[i] = (Ep - Em) * 0.5
        return grad
    def loss_fn(th):
        # real scalar accumulator on the right device/dtype
        energy = torch.zeros((), dtype=th.real.dtype, device=th.device)

        c = build_ansatz_tc_pytorch(th, N, D, use_mps)

        for weight, mask in hm_terms:
            z_idx = [i for i, v in enumerate(mask) if v]
            if z_idx:
                e_term = c.expectation_ps(z=z_idx)     # may be complex64
                e_term = torch.real(e_term)            # keep graph, drop tiny imag part
                w = torch.as_tensor(weight, dtype=energy.dtype, device=th.device)
                energy = energy + w * e_term
            else:
                energy = energy + torch.as_tensor(weight, dtype=energy.dtype, device=th.device)

        return energy  # real torch scalar (required for autograd)

    vg_fn = tc.backend.value_and_grad(loss_fn)
    exp_values: List[float] = []
    best_results: List[Any] = []      # You can push argmin bitstrings here if/when you have them
    iter_times: List[float] = []

    best_energy = float("inf")
    best_any: Any = None

    for step in range(steps):
        t0 = time.time()
        energy = loss_fn(theta)               # scalar (TensorCircuit backend scalar)
        if use_mps:
            grad = parameter_shift_grad(theta)
        else:
          energy, grad  = vg_fn(theta)                 # same shape as theta
        with torch.no_grad():
            theta -= lr * grad
        dt = time.time() - t0
        theta.requires_grad_(True)
        # record
        energy_float = float(np.real(energy))
        exp_values.append(energy_float)
        iter_times.append(dt)
        best_results.append(best_any)         # keep None/placeholder or plug real best if you track it

        if energy_float < best_energy:
            best_energy = energy_float
            best_any = {"step": step, "energy": best_energy}

        #print(f"Step {step:2d}: E={energy_float:.6f} | t={dt:.3f}s")

    # Final θ back to torch tensor for consistency with your RunLog
    final_theta_torch = theta.detach().clone()

    runlog = RunLog(
        exp_values=[float(v) for v in exp_values],
        best_results=list(best_results),
        iter_times=list(iter_times),
        final_theta=final_theta_torch,
    )

    #print(f"⏱️ Avg time/step: {np.mean(iter_times):.3f}s ± {np.std(iter_times):.3f}")

    # save (slim)
    info = None
    if save:
        info = append_run_inplace_slim(
            runlog,
            library=library_name,
            N=N,
            D=D,
            seed = seed,
            outdir=outdir,
            summary_csv=os.path.join(outdir, "summary.csv"),
        )
        print(f"[append] Appended {len(runlog.exp_values)} iters starting from {info['start_iter']} (run_id={info['run_id']}).")

    return (info, runlog) if save else runlog


In [ ]:
# # edges: [((u,v), weight), ...]; graph helper functions assumed available
# graph_lookup = {(g['N'], g['graph_id']): g['G'] for g in graph_by_seed}
# info, log = benchmark_tensorcircuit_tc_pytorch(
#     graph=graph_lookup[(4,0)],
#     D=1,
#     steps=1,
#     use_mps=True,
#     device="cuda",
#     lr=0.05,
#     library_name="tensorcircuit_cpu",
#     save=True,
#     outdir="bench_out_comparison"
# )


In [ ]:
import time
import numpy as np
import torch
import pennylane as qml
from typing import Any, List, Optional

# expects you already have:
# - RunLog dataclass
# - append_run_inplace_slim(...)
# - create_hamiltonian(...) and make_hamiltonian(...), sparse_pauliop_to_qml_hamiltonian(...)
def sparse_pauliop_to_qml_hamiltonian(sparse_op):
    """Convert Qiskit SparsePauliOp to PennyLane Hamiltonian."""
    coeff_list = sparse_op.coeffs.real.tolist()
    op_list = []

    for pauli_str in sparse_op.paulis.to_labels():
        terms = []
        for idx, pauli_char in enumerate(pauli_str):
            idx = len(pauli_str)-1-idx
            if pauli_char == "I":
                continue
            elif pauli_char == "X":
                terms.append(qml.PauliX(idx))
            elif pauli_char == "Y":
                terms.append(qml.PauliY(idx))
            elif pauli_char == "Z":
                terms.append(qml.PauliZ(idx))
        if terms:
            op = terms[0]
            for term in terms[1:]:
                op = op @ term
        else:
            op = qml.Identity(0)
        op_list.append(op)

    return qml.Hamiltonian(coeff_list, op_list)
def _qml_hamiltonian_from_graph(graph) -> Optional[qml.Hamiltonian]:
    """
    Try to build a PennyLane Hamiltonian from your helpers.
    Fallback to Z0 if helpers are missing.
    """
    H_sparse = make_hamiltonian(graph)  # your SparsePauliOp (or similar)
    return sparse_pauliop_to_qml_hamiltonian(H_sparse)

def _pick_pl_device(device: str) -> str:
    """
    Map torch device string to a PennyLane device name.
    Prefer lightning.gpu for cuda; fall back gracefully.
    """
    if device == "cuda":
        try:
            # Try GPU device if available in your env
            _ = qml.device("lightning.gpu", wires=1)  # probe
            return "lightning.gpu"
        except Exception:
            return "default.qubit"
    else:
        # CPU
        try:
            _ = qml.device("lightning.qubit", wires=1)
            return "lightning.qubit"
        except Exception:
            return "default.qubit"

def benchmark_pennylane_device(
    graph,
    theta,
    D: int,
    seed: int,
    steps: int,
    use_mps: bool,            # kept for signature compatibility (unused in PL)
    device: str,              # "cpu" or "cuda"
    lr: float = 0.1,
    *,
    library_name: str = "pennylane",
    save: bool = True,
    outdir: str = "bench_out_pl"
):
    """
    PennyLane benchmark with TensorCircuit-compatible signature.
    Returns (info_dict_or_None, RunLog).
    """
    # infer N from edges: graph like [((u,v), w), ...]
    N = max(max(u, v) for (u, v), _ in graph) + 1
    #pl_dev = _pick_pl_device(device)
    pl_dev = device
    graph = create_hamiltonian(N,graph)
    print(f"\n⚙️ PennyLane VQE | N={N} | dev={pl_dev} | torch={device} | D={D} | steps={steps}")

    # Try to use your true Hamiltonia n; fallback to Z0
    H_qml = _qml_hamiltonian_from_graph(graph)
    # if H_qml is None:
    #     H_qml = qml.Hamiltonian([1.0], [qml.PauliZ(0)])
    #     print("  (ℹ️ using fallback H = Z0)")

    dev_pl = qml.device(pl_dev, wires=N)
    @qml.qnode(dev_pl, interface="torch", diff_method="parameter-shift")
    def circuit(theta):
        # theta shape: (D, 2N)
        for d in range(D):
            for j in range(N):
                qml.RX(theta[d, j], wires=j)
            for j in range(N):
                qml.RZ(theta[d, N + j], wires=j)
            for j in range(N - 1):
                qml.CNOT(wires=[j, j + 1])
            qml.CNOT(wires=[N-1, 0])
        return qml.expval(H_qml)

    # θ on requested torch device
    # Slice to first 8 elements
    theta = theta[:D* 2 * N]  # shape: [8]

    # Reshape to [2, 4]
    theta = theta.view(D, 2*N).clone().detach().requires_grad_(True)
    #print(theta)
    #theta = torch.rand((D, 2 * N), dtype=torch.float32, requires_grad=True, device=device)
    optimizer = torch.optim.SGD([theta], lr=lr)

    exp_values: List[float] = []
    best_results: List[Any] = []
    iter_times: List[float] = []

    best_energy = float("inf")
    best_any: Any = None

    for step in range(steps):
        t0 = time.time()

        optimizer.zero_grad(set_to_none=True)
        energy = circuit(theta)  # torch scalar
        energy.backward()
        optimizer.step()

        dt = time.time() - t0
        e_val = float(energy.detach().cpu().item())

        exp_values.append(e_val)
        iter_times.append(dt)
        best_results.append(best_any)

        if e_val < best_energy:
            best_energy = e_val
            best_any = {"step": step, "energy": best_energy}

        #print(f"Step {step:2d}: E = {e_val:.6f} | t = {dt:.4f}s")

    final_theta_cpu = theta.detach().to("cpu").clone()
    runlog = RunLog(
        exp_values=[float(v) for v in exp_values],
        best_results=list(best_results),
        iter_times=list(iter_times),
        final_theta=final_theta_cpu,
    )

    # optional: persist with slim logger (keyed only by library, N, D)
    info = None
    if save:
        info = append_run_inplace_slim(
            runlog,
            library=f"{library_name}:{pl_dev}",
            N=N,
            D=D,
            seed=seed,
            outdir=outdir,
            summary_csv=f"{outdir}/summary.csv",
        )
        print(f"[append] Appended {len(runlog.exp_values)} iters starting from {info['start_iter']} (run_id={info['run_id']}).")

    return info, runlog


/usr/local/lib/python3.12/dist-packages/pennylane/__init__.py:209: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.6.2. You have version 0.7.2 installed. Please downgrade JAX to 0.6.2 to avoid runtime errors using python -m pip install jax~=0.6.0 jaxlib~=0.6.0
  warnings.warn(


In [ ]:
import time
import numpy as np
import torch
from typing import Any, List
import numpy as np
import time
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.quantum_info import SparsePauliOp
lr =1e-2
def build_ansatz_qiskit(theta, N, D):
    qc = QuantumCircuit(N)
    for d in range(D):
        for j in range(N):
            qc.rx(theta[d, j], j)
        for j in range(N):
            qc.rz(theta[d, N + j], j)
        for j in range(N - 1):
            qc.cx(j, j + 1)
        qc.cx(N-1,0)
    return qc
def expectation_qiskit(theta, N, D, obs, use_mps, device='CPU'):
    qc = build_ansatz_qiskit(theta, N, D)
    sim = AerSimulator(method="matrix_product_state" if use_mps else "statevector", device=device)
    qc.save_statevector()
    result = sim.run(qc).result()
    sv = result.get_statevector(qc)
    return np.real(sv.expectation_value(obs))

def parameter_shift_grad(theta, N, D, obs, shift=np.pi/2, use_mps=False, device='CPU'):
    grads = np.zeros_like(theta)
    for i in range(D):
        for j in range(2 * N):
            theta_plus = theta.copy(); theta_plus[i, j] += shift
            theta_minus = theta.copy(); theta_minus[i, j] -= shift
            f_plus = expectation_qiskit(theta_plus, N, D, obs, use_mps, device)
            f_minus = expectation_qiskit(theta_minus, N, D, obs, use_mps, device)
            grads[i, j] = 0.5 * (f_plus - f_minus)
    return grads
# expects:
# - RunLog dataclass
# - append_run_inplace_slim(library, N, D, outdir, summary_csv)
# - make_hamiltonian(graph) -> observable for Qiskit
# - expectation_qiskit(theta, N, D, obs, use_mps, device) -> float
# - parameter_shift_grad(theta, N, D, obs, use_mps, device) -> np.ndarray same shape as theta

def benchmark_qiskit_device(
    graph,
    theta,
    D: int,
    seed: int,
    steps: int,
    use_mps: bool,
    device: str,               # e.g. "CPU" | "GPU" | backend name you handle internally
    lr: float = 0.1,
    *,
    library_name: str = "qiskit",
    save: bool = True,
    outdir: str = "bench_out_qiskit",
):
    """
    Qiskit benchmark with the SAME signature as your TensorCircuit function.
    Returns (info_dict_or_None, RunLog).
    """
    # infer N from edges: graph like [((u,v), w), ...]
    N = max(max(u, v) for (u, v), _ in graph) + 1
    backend_tag = "mps" if use_mps else "statevector"
    print(f"\n🔧 Qiskit {backend_tag.upper()} | N={N} | Device={device} | D={D} | steps={steps}")

    # Hamiltonian from your helper
    obs = create_hamiltonian(N,graph)
    obs = make_hamiltonian(obs)

    # θ ∈ R^{D × 2N}
    theta = theta[:D* 2 * N]  # shape: [8]

    # Reshape to [2, 4]
    theta = theta.detach().cpu().numpy().reshape(D, 2 * N).astype(np.float64)
    #theta = np.random.rand(D, 2 * N).astype(np.float64)

    exp_values: List[float] = []
    best_results: List[Any] = []
    iter_times: List[float] = []

    best_energy = float("inf")
    best_any: Any = None

    for step in range(steps):
        t0 = time.time()

        energy = expectation_qiskit(theta, N, D, obs, use_mps=use_mps, device=device)
        grad   = parameter_shift_grad(theta, N, D, obs, use_mps=use_mps, device=device)
        theta -= lr * grad

        dt = time.time() - t0
        exp_values.append(float(energy))
        iter_times.append(dt)
        best_results.append(best_any)

        if energy < best_energy:
            best_energy = float(energy)
            best_any = {"step": step, "energy": best_energy}

        #print(f"Step {step:2d}: E={float(energy):.6f} | t={dt:.3f}s")

    # Pack as RunLog (final theta as torch tensor for consistency)
    final_theta_torch = torch.from_numpy(theta.copy()).detach().clone()
    runlog = RunLog(
        exp_values=[float(v) for v in exp_values],
        best_results=list(best_results),
        iter_times=list(iter_times),
        final_theta=final_theta_torch,
    )

    # Optional: persist with slim logger keyed only by (library, N, D)
    info = None
    if save:
        info = append_run_inplace_slim(
            runlog,
            library=f"{library_name}:{backend_tag}",
            N=N,
            D=D,
            seed=seed,
            outdir=outdir,
            summary_csv=f"{outdir}/summary.csv",
        )
        print(f"[append] Appended {len(runlog.exp_values)} iters starting from {info['start_iter']} (run_id={info['run_id']}).")

    return info, runlog


In [ ]:
import time
import copy
from typing import Any, List

import numpy as np
import torch

import quimb as qu
import quimb.tensor as qtn

# expects you already have:
# - RunLog dataclass
# - append_run_inplace_slim(library, N, D, outdir, summary_csv)
# - create_hamiltonian(N, graph) -> list[(weight, mask)] with mask: List[bool] length N
#   (or pass directly the same "graph" format you used elsewhere and call create_hamiltonian before)

# ────────────────────────────────────────────────────────────────────────────────
# Helpers
# ────────────────────────────────────────────────────────────────────────────────

def _build_ansatz_quimb(theta: np.ndarray, N: int, D: int) -> qtn.Circuit:
    """
    theta: shape (D, 2N)
    Returns a quimb Circuit with layers:
      [ RX(j), RZ(j) for j in 0..N-1 ] + CNOT chain (j->j+1)
    """
    c = qtn.Circuit(N)
    for d in range(D):
        # RX on all qubits
        for j in range(N):
            c.apply_gate("RX", float(theta[d, j]), j)
        # RZ on all qubits
        for j in range(N):
            c.apply_gate("RZ", float(theta[d, N + j]), j)
        # linear entangling CNOTs
        for j in range(N - 1):
            c.apply_gate("CNOT", j, j + 1)
        c.apply_gate("CNOT", N-1,0)
    return c

def _psi_probs_from_circuit(circ: qtn.Circuit) -> np.ndarray:
    """
    Return |psi|^2 probabilities as a 1D array of length 2**N.
    """
    # get the state as a TensorNetwork ket and densify
    psi = circ.psi  # TN object
    vec = psi.to_dense()  # complex ndarray shape (2**N,)
    probs = (vec.real**2 + vec.imag**2).astype(np.float64)
    return probs

def _z_product_expectation_from_probs(probs: np.ndarray, N: int, z_idx: List[int]) -> float:
    """
    E[Z_{z1} Z_{z2} ...] computed from computational basis probabilities.
    sign = (-1)^{# of 1-bits among z_idx in basis state}
    """
    if not z_idx:
        return 1.0
    # Precompute bit masks
    mask = 0
    for i in z_idx:
        mask |= (1 << i)
    exp_val = 0.0
    for b, p in enumerate(probs):
        # count ones among z_idx: popcount of (b & mask)
        ones = (b & mask).bit_count()
        sign = -1.0 if (ones % 2) else 1.0
        exp_val += sign * p
    return float(exp_val)

def _energy_from_theta(theta: np.ndarray, N: int, D: int, hm_terms: List[tuple]) -> float:
    """
    hm_terms: iterable of (weight, mask) where mask is List[bool] length N (Z on True sites).
    Energy = sum_i -w_i * <Z^mask_i>
    """
    circ = _build_ansatz_quimb(theta, N, D)
    probs = _psi_probs_from_circuit(circ)
    bitstrings = [format(i, f'0{N}b') for i in range(2**N)]
    prob_dict = {b: float(val) for b, val in zip(bitstrings, probs.flatten())}
    #print(prob_dict)
    E = 0.0

    for w, mask in hm_terms:
        mask = mask[::-1]
        z_idx = [i for i, m in enumerate(mask) if m]

        #if z_idx:
        E += float(w) * _z_product_expectation_from_probs(probs, N, z_idx)

    return float(E)

def _parameter_shift_grad(theta: np.ndarray, N: int, D: int, hm_terms: List[tuple]) -> np.ndarray:
    """
    Parameter-shift gradient for RX/RZ gates: shift ±π/2.
    Returns grad with same shape as theta.
    """
    grad = np.zeros_like(theta, dtype=np.float64)
    shift = np.pi / 2.0

    # Loop over all parameters
    for d in range(D):
        for k in range(2 * N):
            # + shift
            th_plus = theta.copy()
            th_plus[d, k] += shift
            E_plus = _energy_from_theta(th_plus, N, D, hm_terms)
            # - shift
            th_minus = theta.copy()
            th_minus[d, k] -= shift
            E_minus = _energy_from_theta(th_minus, N, D, hm_terms)

            grad[d, k] = 0.5 * (E_plus - E_minus)
    return grad
import numpy as np
import quimb as qu

def quimb_H_from_bool_terms(hm_terms, N=None):
    """
    hm_terms: list[(weight: float, mask: tuple[bool,...])]
      True -> Z, False -> I
    N: number of qubits; inferred from first mask if None.
    """
    if N is None:
        N = len(hm_terms[0][1])
    N = int(N)

    I = qu.pauli('I')
    Z = qu.pauli('Z')

    H = np.zeros((2**N, 2**N), dtype=complex)

    for w, mask in hm_terms:
        # collect non-identity sites
        inds = [i for i, b in enumerate(mask) if b]
        if not inds:
            H += w * np.eye(2**N, dtype=complex)
            continue

        # kron only the Zs, then place them at indices via ikron
        term_local = Z
        for _ in range(len(inds) - 1):
            term_local = qu.kron(term_local, Z)

        term_full = qu.ikron(term_local, dims=[2]*N, inds=inds)
        H += w * term_full

    return qu.qarray(H)

def norm_fn(psi):
    # we could always define this within the loss function, but separating it
    # out can be clearer - it's also called before returning the optimized TN
    nfact = (psi.H @ psi)**0.5
    return psi.multiply(1 / nfact, spread_over='all')


def loss_fn(psi, ham):
    b, h, k = qtn.tensor_network_align(psi.H, ham, psi)
    energy_tn = b | h | k
    return energy_tn ^ ...
# ────────────────────────────────────────────────────────────────────────────────
# Main: same signature as your other backends
# ────────────────────────────────────────────────────────────────────────────────

def benchmark_quimb_device(
    graph,
    theta,
    D: int,
    seed: int,
    steps: int,
    use_mps: bool,         # accepted for signature compatibility; not used in this baseline
    device: str,           # "cpu" or "cuda" (quimb here runs on CPU; kept for signature symmetry)
    lr: float = 0.1,
    *,
    library_name: str = "quimb",
    save: bool = True,
    outdir: str = "bench_out_quimb",
):
    """
    Quimb benchmark with TensorCircuit-compatible signature.
    Uses quimb.tensor.Circuit for the ansatz and parameter-shift gradients.
    Returns (info_dict_or_None, RunLog).
    """
    # infer N from edges: graph like [((u,v), w), ...]
    N = max(max(u, v) for (u, v), _ in graph) + 1
    print(f"\n🧵 Quimb | N={N} | backend={'MPS' if use_mps else 'TN/Circuit'} | device={device} | D={D} | steps={steps}")

    # Build Ising-like terms from your helper (same as your TensorCircuit code)
    hm_terms = create_hamiltonian(N, graph)  # -> list of (weight, mask)
    print(hm_terms)
    # θ ∈ R^{D × 2N}
    # rng = np.random.default_rng()
    # theta = rng.standard_normal((D, 2 * N)).astype(np.float64)
        # θ ∈ R^{D × 2N}
    theta = theta[:D* 2 * N]  # shape: [8]

    # Reshape to [2, 4]
    theta = theta.detach().cpu().numpy().reshape(D, 2 * N).astype(np.float64)
    #theta = np.random.rand(D, 2 * N).astype(np.float64)
    exp_values: List[float] = []
    best_results: List[Any] = []
    iter_times: List[float] = []
    circ = _build_ansatz_quimb(theta, N, D)
    best_energy = float("inf")
    best_any: Any = None

    ham = quimb_H_from_bool_terms(hm_terms, N )
    tnopt = qtn.TNOptimizer(
        # the tensor network we want to optimize
        circ,
        # the functions specfying the loss and normalization
        loss_fn=loss_fn,
        norm_fn=norm_fn,
        # we specify constants so that the arguments can be converted
        # to the  desired autodiff backend automatically
        loss_constants={"ham": ham},
        # the underlying algorithm to use for the optimization
        # 'l-bfgs-b' is the default and often good for fast initial progress
        optimizer="sdg",
        # which gradient computation backend to use
        autodiff_backend="jax",
    )
    # for step in range(steps):
    #     t0 = time.time()

    #     # energy and gradient via parameter shift
    #     energy = _energy_from_theta(theta, N, D, hm_terms)
    #     #grad = _parameter_shift_grad(theta, N, D, hm_terms)
    #     grad =
    #     theta = theta - lr * grad

    #     dt = time.time() - t0
    #     exp_values.append(float(energy))
    #     iter_times.append(dt)
    #     best_results.append(best_any)

    #     if energy < best_energy:
    #         best_energy = float(energy)
    #         best_any = {"step": step, "energy": best_energy}

    #     #print(f"Step {step:2d}: E={float(energy):.6f} | t={dt:.3f}s")

    final_theta_torch = torch.from_numpy(theta.copy()).detach().clone()
    runlog = RunLog(
        exp_values=[float(v) for v in exp_values],
        best_results=list(best_results),
        iter_times=list(iter_times),
        final_theta=final_theta_torch,
    )

    info = None
    if save:
        info = append_run_inplace_slim(
            runlog,
            library=f"{library_name}:{'mps' if use_mps else 'tn'}",
            N=N,
            D=D,
            seed=seed,
            outdir=outdir,
            summary_csv=f"{outdir}/summary.csv",
        )
        print(f"[append] Appended {len(runlog.exp_values)} iters starting from {info['start_iter']} (run_id={info['run_id']}).")

    return info, runlog


In [ ]:
target_N = 52
graph_lookup = {(g['N'], g['graph_id']): g['G'] for g in graph_by_seed}
for N in [24]:
  for D in [3]:
      for seed in [0]:
        num_theta = 2*N*D
        graph = graph_lookup[(N,seed)]
        theta = theta_init_by_seed[seed][:num_theta]
        steps = 10
        # info, log = benchmark_tensorcircuit_tc_pytorch(
        #     graph=graph,
        #     D=D,
        #     seed = seed,
        #     steps=steps,
        #     use_mps=False,
        #     device="cuda",
        #     theta=theta,
        #     lr=0.05,
        #     library_name="tensorcircuit_full",
        #     save=True,
        #     outdir="bench_out_comparison_cpu_real"
        # )
        # info, log = benchmark_tensorcircuit_tc_pytorch(
        #     graph=graph,
        #     D=D,
        #     seed = seed,
        #     steps=steps,
        #     use_mps=True,
        #     device="cuda",
        #     theta=theta,
        #     lr=0.05,
        #     library_name="tensorcircuit_mps",
        #     save=True,
        #     outdir="bench_out_comparison_cpu_real"
        # )

        # info, log = benchmark_pennylane_device(
        #     graph=graph,      # [((u,v), w), ...]
        #     theta=theta,
        #     D=D,
        #     seed=seed,
        #     steps=steps,
        #     use_mps=True,     # accepted but ignored by PL
        #     device="default.qubit",    # or "cpu"
        #     lr=0.05,
        #     library_name="pennylane_default",
        #     save=True,
        #     outdir="bench_out_comparison_cpu_real"
        # )
        # info, log = benchmark_pennylane_device(
        #     graph=graph,      # [((u,v), w), ...]
        #     theta=theta,
        #     D=D,
        #     seed=seed,
        #     steps=steps,
        #     use_mps=True,     # accepted but ignored by PL
        #     device="lightning.qubit",    # or "cpu"
        #     lr=0.05,
        #     library_name="pennylane_lightning",
        #     save=True,
        #     outdir="bench_out_comparison_cpu_real"
        # )
        info, log = benchmark_qiskit_device(
          graph=graph,      # [((u,v), w), ...]\
          theta = theta_init_by_seed[0],
          D=D,
          seed=seed,
          steps=steps,
          use_mps=True,     # accepted but ignored by PL
          device="CPU",    # or "cpu"
          lr=0.05,
          library_name="qiskit_mps",
          save=True,
          outdir="bench_out_comparison_cpu_real"
        )
        # info, log = benchmark_quimb_device(
        #   graph=graph,      # [((u,v), w), ...]\
        #   theta = theta_init_by_seed[0],
        #   D=D,
        #   seed=seed,
        #   steps=steps,
        #   use_mps=True,     # accepted but ignored by PL
        #   device="CPU",    # or "cpu"
        #   lr=0.05,
        #   library_name="quimb",
        #   save=True,
        #   outdir="bench_out_comparison_cpu_real"
        # )




🔧 Qiskit MPS | N=24 | Device=CPU | D=3 | steps=10


KeyboardInterrupt: 

In [ ]:
target_N = 52
graph_lookup = {(g['N'], g['graph_id']): g['G'] for g in graph_by_seed}
for N in [4,8,16]:
  for D in [3]:
      for seed in [1,2]:
        num_theta = 2*N*D
        graph = graph_lookup[(N,seed)]
        theta = theta_init_by_seed[seed][:num_theta]
        steps = 100
        info, log = benchmark_tensorcircuit_tc_pytorch(
            graph=graph,
            D=D,
            seed = seed,
            steps=steps,
            use_mps=False,
            device="cuda",
            theta=theta,
            lr=0.05,
            library_name="tensorcircuit_full",
            save=True,
            outdir="bench_out_comparison_cpu_real"
        )
        info, log = benchmark_tensorcircuit_tc_pytorch(
            graph=graph,
            D=D,
            seed = seed,
            steps=steps,
            use_mps=True,
            device="cuda",
            theta=theta,
            lr=0.05,
            library_name="tensorcircuit_mps",
            save=True,
            outdir="bench_out_comparison_cpu_real"
        )

        info, log = benchmark_pennylane_device(
            graph=graph,      # [((u,v), w), ...]
            theta=theta,
            D=D,
            seed=seed,
            steps=steps,
            use_mps=True,     # accepted but ignored by PL
            device="default.qubit",    # or "cpu"
            lr=0.05,
            library_name="pennylane_default",
            save=True,
            outdir="bench_out_comparison_cpu_real"
        )
        info, log = benchmark_pennylane_device(
            graph=graph,      # [((u,v), w), ...]
            theta=theta,
            D=D,
            seed=seed,
            steps=steps,
            use_mps=True,     # accepted but ignored by PL
            device="lightning.qubit",    # or "cpu"
            lr=0.05,
            library_name="pennylane_lightning",
            save=True,
            outdir="bench_out_comparison_cpu_real"
        )
        info, log = benchmark_qiskit_device(
          graph=graph,      # [((u,v), w), ...]\
          theta = theta_init_by_seed[0],
          D=D,
          seed=seed,
          steps=steps,
          use_mps=True,     # accepted but ignored by PL
          device="CPU",    # or "cpu"
          lr=0.05,
          library_name="qiskit_mps",
          save=True,
          outdir="bench_out_comparison_cpu_real"
        )
        info, log = benchmark_quimb_device(
          graph=graph,      # [((u,v), w), ...]\
          theta = theta_init_by_seed[0],
          D=D,
          seed=seed,
          steps=steps,
          use_mps=True,     # accepted but ignored by PL
          device="CPU",    # or "cpu"
          lr=0.05,
          library_name="quimb",
          save=True,
          outdir="bench_out_comparison_cpu_real"
        )

